In [17]:
!pip install keybert sentence-transformers nltk pandas
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [13]:
import pandas as pd

# Correct URL for the BBC text classification dataset
!wget https://huggingface.co/datasets/SetFit/bbc-news/resolve/main/bbc-text.csv -O bbc-text.csv

--2025-07-16 22:12:57--  https://huggingface.co/datasets/SetFit/bbc-news/resolve/main/bbc-text.csv
Resolving huggingface.co (huggingface.co)... 3.166.152.110, 3.166.152.44, 3.166.152.65, ...
Connecting to huggingface.co (huggingface.co)|3.166.152.110|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: /api/resolve-cache/datasets/SetFit/bbc-news/9c31f7087a8c225be8b4c420d70216ef1f121eb0/bbc-text.csv?%2Fdatasets%2FSetFit%2Fbbc-news%2Fresolve%2Fmain%2Fbbc-text.csv=&etag=%22b91753711617669f13c9d9c994496ec63aba8130%22 [following]
--2025-07-16 22:12:57--  https://huggingface.co/api/resolve-cache/datasets/SetFit/bbc-news/9c31f7087a8c225be8b4c420d70216ef1f121eb0/bbc-text.csv?%2Fdatasets%2FSetFit%2Fbbc-news%2Fresolve%2Fmain%2Fbbc-text.csv=&etag=%22b91753711617669f13c9d9c994496ec63aba8130%22
Reusing existing connection to huggingface.co:443.
HTTP request sent, awaiting response... 200 OK
Length: 5057493 (4.8M) [text/plain]
Saving to: ‘bbc-text.csv’

bbc-tex

In [14]:
import pandas as pd

df = pd.read_csv("bbc-text.csv")
print(df.columns)           # Should show 'category', 'text'
texts = df['text'].fillna("").tolist()
print("Loaded", len(texts), "articles. Example:\n", texts[:2])


Index(['category', 'text'], dtype='object')
Loaded 2225 articles. Example:
 ['tv future in the hands of viewers with home theatre systems  plasma high-definition tvs  and digital video recorders moving into the living room  the way people watch tv will be radically different in five years  time.  that is according to an expert panel which gathered at the annual consumer electronics show in las vegas to discuss how these new technologies will impact one of our favourite pastimes. with the us leading the trend  programmes and other content will be delivered to viewers via home networks  through cable  satellite  telecoms companies  and broadband service providers to front rooms and portable devices.  one of the most talked-about technologies of ces has been digital and personal video recorders (dvr and pvr). these set-top boxes  like the us s tivo and the uk s sky+ system  allow people to record  store  play  pause and forward wind tv programmes when they want.  essentially  the technolo

In [15]:
from keybert import KeyBERT
from nltk.corpus import wordnet as wn
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
kw_model = KeyBERT(model=embedding_model)

# Extract keywords
def extract_keywords(text, top_n=5):
    return [kw[0] for kw in kw_model.extract_keywords(text, top_n=top_n)]

# Expand keywords using WordNet
def expand_keywords(keywords):
    expanded = set(keywords)
    for kw in keywords:
        for syn in wn.synsets(kw):
            for lemma in syn.lemmas():
                word = lemma.name().replace("_", " ").lower()
                if word != kw:
                    expanded.add(word)
    return list(expanded)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [20]:
from nltk.tokenize import word_tokenize
from collections import Counter

def evaluate_expansion(text, original_keywords, expanded_keywords):
    tokens = [w.lower() for w in word_tokenize(text) if w.isalpha()]
    common = [w for w, _ in Counter(tokens).most_common(20)]

    original_covered = len(set(original_keywords) & set(common)) / len(common)
    expanded_covered = len(set(expanded_keywords) & set(common)) / len(common)

    return original_covered, expanded_covered

# Run on sample
text = texts[1] # Changed to use the second text
original = extract_keywords(text, top_n=10)
expanded = expand_keywords(original)
orig_cov, exp_cov = evaluate_expansion(text, original, expanded)

print("Original Keywords:", original)
print("Expanded Keywords:", expanded[:10])
print(f"\nCoverage before: {orig_cov:.2f}, after expansion: {exp_cov:.2f}")

Original Keywords: ['fraud', 'worldcom', 'accounting', 'accused', 'prosecutors', 'ceo', 'ebbers', 'guilty', 'testify', 'myers']
Expanded Keywords: ['pseudo', 'prosecuting attorney', 'accounting', 'impostor', 'calculate', 'role player', 'account statement', 'shamefaced', 'shammer', 'incriminate']

Coverage before: 0.20, after expansion: 0.20


In [21]:
# Evaluate Diversity of Keywords
original_unique_count = len(set(original))
expanded_unique_count = len(set(expanded))

print(f"Number of unique original keywords: {original_unique_count}")
print(f"Number of unique expanded keywords: {expanded_unique_count}")

# You can also look at the keywords themselves to subjectively assess diversity
print("\nOriginal Keywords:", original)
print("Expanded Keywords (first 20):", list(expanded)[:20])

Number of unique original keywords: 10
Number of unique expanded keywords: 54

Original Keywords: ['fraud', 'worldcom', 'accounting', 'accused', 'prosecutors', 'ceo', 'ebbers', 'guilty', 'testify', 'myers']
Expanded Keywords (first 20): ['pseudo', 'prosecuting attorney', 'accounting', 'impostor', 'calculate', 'role player', 'account statement', 'shamefaced', 'shammer', 'incriminate', 'imposter', 'answer for', 'hangdog', 'prosecutor', 'accused', 'testify', 'pseud', 'faker', 'fraud', 'humbug']


Great! Let's start by looking at the diversity of the keywords. A simple way to measure diversity is to count the number of unique words in the original and expanded keyword lists. I'll add some code to calculate and display this.

We've now evaluated the diversity of the keywords by looking at the number of unique terms before and after expansion. The output shows that the expanded keywords have significantly more unique terms, suggesting increased diversity.

In [22]:
from sentence_transformers import util

# Get the embedding for the entire text
text_embedding = embedding_model.encode(text, convert_to_tensor=True)

# Get embeddings for the expanded keywords
expanded_keyword_embeddings = embedding_model.encode(expanded, convert_to_tensor=True)

# Calculate the average embedding of the expanded keywords
average_expanded_keyword_embedding = expanded_keyword_embeddings.mean(dim=0)

# Calculate the cosine similarity between the text embedding and the average expanded keyword embedding
similarity = util.cos_sim(text_embedding, average_expanded_keyword_embedding)

print(f"Cosine similarity between text and average expanded keyword embedding: {similarity.item():.4f}")

Cosine similarity between text and average expanded keyword embedding: 0.3844


Yes, we can explore other metrics without having to read each article manually. One approach is to look at the semantic similarity between the expanded keywords and the article text itself.

We can calculate the embedding for the entire article and the embeddings for the expanded keywords, then measure the similarity between them. A higher similarity might indicWe've now calculated the cosine similarity between the embedding of the article text and the average embedding of the expanded keywords.

This similarity score (0.3844) is one way to quantify how semantically close the expanded keywords are to the overall meaning of the article. A higher score would generally suggest that the expanded keywords are more relevant to the text.

We can interpret this score as a measure of how well the expanded keywords, as a group, represent the central theme or meaning of the article, based on the embedding model's understanding of semantics.ate that the expanded keywords are more representative of the article's content.

In [23]:
# Get embeddings for the original keywords
original_keyword_embeddings = embedding_model.encode(original, convert_to_tensor=True)

# Calculate the average embedding of the original keywords
average_original_keyword_embedding = original_keyword_embeddings.mean(dim=0)

# Calculate the cosine similarity between the text embedding and the average original keyword embedding
original_similarity = util.cos_sim(text_embedding, average_original_keyword_embedding)

print(f"Cosine similarity between text and average original keyword embedding: {original_similarity.item():.4f}")
print(f"Cosine similarity between text and average expanded keyword embedding: {similarity.item():.4f}") # Display expanded similarity again for comparison

Cosine similarity between text and average original keyword embedding: 0.5431
Cosine similarity between text and average expanded keyword embedding: 0.3844


You can see from the output that the cosine similarity between the text and the average original keyword embedding is 0.5431, while the similarity with the average expanded keyword embedding is 0.3844.

In this particular case, the average embedding of the original keywords is more similar to the overall text embedding than the average embedding of the expanded keywords. This suggests that for this specific article and the current expansion method, the original keywords are, on average, more semantically aligned with the text's main theme according to the embedding model.

This doesn't necessarily mean the expansion is "bad," but it indicates that the words added through expansion might be semantically further away from the core topic of the text compared to the initial keywords.